These exercises were completed as part of DataCamp's "Intro to Deep Learning with Keras" course.

# Detecting counterfeit banknotes: binary classification

In [1]:
import pandas as pd

In [6]:
x = 

In [7]:
x = x.astype('int32')

AttributeError: 'list' object has no attribute 'astype'

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K

In [ ]:
plt.rcParams['figure.figsize'] = (8, 8)

In [ ]:
banknotes = pd.read_csv('banknotes.csv')
banknotes.head()

### Look at the data

In [ ]:
banknotes.head()

In [ ]:
# Describe the data
print('Dataset stats: \n', banknotes.describe())

In [ ]:
# Count the number of observations per class
print('Observations per class: \n', banknotes['class'].value_counts())

**Note:** `0` indicates a bill is **real** and `1` indicates a bill is **fake**.

Task: Perform binary classification by using a single neuron as an output. The input layer will have 4 neurons since there are 4 features in our dataset. The model's output will be a value constrained between 0 and 1.

### Create the model

In [ ]:
# Create a sequential model
model = tf.keras.Sequential()

# Add a dense layer
model.add(tf.keras.layers.Dense(1, input_shape=(4, ), activation='sigmoid'))

# Compile your model
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Display a summary of your model
model.summary()

### Prepare the dataset
First, split the dataset into features (`X`) and labels (`y`). Then scale the data and convert it to a form that the model can work with. Finally, split the data into training and testing sets so that we can evaluate our model once it has been trained.

In [ ]:
# Split dataset into features and labels
X = banknotes.iloc[:, :4]
y = banknotes['class'].to_numpy()

In [ ]:
# Scale relevant data (X, features)
X = ((X - X.mean()) / X.std()).to_numpy()

Take a look at the data with Seaborn's pairplot before feeding splitting it into training and testing sets and feeding it into the model.

In [ ]:
sns.pairplot(banknotes, hue='class')
plt.show()

In [ ]:
# Split features and labels into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

### Train the model
In only 1 line of code!

In [ ]:
model.fit(X_train, y_train, epochs=20)

### Is this dollar bill fake ?
Evaluate the model with the test set.

In [ ]:
# Evaluate your model accuracy on the test set
accuracy = model.evaluate(X_test, y_test)[1]

# Print accuracy
print('Accuracy: ', accuracy)

**Conclusion:** The model did pretty well, with an accuracy of 90%! In real-life scenarios, (and with a bit more data), it would be preferrable to do both a train test split, as well as a validation split, to ensure there isn't overfitting. For demonstration purposes though, this model worked well!

### Early Stopping

In [ ]:
# Define a callback to monitor val_acc
monitor_val_acc = EarlyStopping(monitor='val_accuracy', 
                       patience=5)

# Train your model using the early stopping callback
model.fit(X_train, y_train, 
           epochs=1000, validation_data=(X_test, y_test),
           callbacks= [monitor_val_acc])

### A combination of callbacks
Deep learning models can take a long time to train, especially when you move to deeper architectures and bigger datasets. Saving your model every time it improves as well as stopping it when it no longer does allows you to worry less about choosing the number of epochs to train for. You can also restore a saved model anytime and resume training where you left it.

Now use both the `EarlyStopping()` *and* the `ModelCheckpoint()` callbacks.

In [ ]:
# Early stop on validation accuracy
monitor_val_acc = EarlyStopping(monitor = 'val_accuracy', patience = 3)

# Save the best model as best_banknote_model.hdf5
modelCheckpoint = ModelCheckpoint('best_banknote_model.hdf5', save_best_only = True)

# Fit your model for a stupid amount of epochs
h_callback = model.fit(X_train, y_train,
                    epochs = 1000000000000,
                    callbacks = [monitor_val_acc, modelCheckpoint],
                    validation_data = (X_test, y_test))

In [ ]:
!ls | grep best_banknote*

**Our final model not only has a validation set accuracy of .9825, but was fit in under a second!**

If you have already built a model, you can use the `model.layers` and the `tf.keras.backend` to build functions that, provided with a valid input tensor, return the corresponding output tensor.

This is a useful tool when we want to obtain the output of a network at an intermediate layer.

For instance, if you get the input and output from the first layer of a network, you can build an `inp_to_out` function that returns the result of carrying out forward propagation through only the first layer for a given input tensor.

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(2, input_shape=(4, ), activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# Input tensor from the 1st layer of the model
inp = model.layers[0].input

# Output tensor from the 1st layer of the model
out = model.layers[0].output

# Define a function from inputs to outputs
inp_to_out = K.function([inp], [out])

# Print the results of passing X_test through the 1st layer
print(inp_to_out([X_test]))

### Neural separation
Neurons learn by updating their weights to output values that help them better distinguish between the different output classes in your dataset. You will make use of the `inp_to_out()` function you just built to visualize the output of two neurons in the first layer of the Banknote Authentication `model` as it learns.

In [ ]:
def plot():
    fig, axes = plt.subplots(1, 5, figsize=(16, 8))
    for i, a in enumerate(axes):
        a.scatter(layer_outputs[i][:, 0], layer_outputs[i][:, 1], c=y_test, edgecolors='none');
        a.set_title('Test Accuracy: {:3.1f} %'.format(float(test_accuracies[i]) * 100.));
    plt.tight_layout()

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
layer_outputs = []
test_accuracies = []

for i in range(0, 21):
    # Train model for 1 epoch
    h = model.fit(X_train, y_train, batch_size=16, epochs=1, verbose=0)
    if i % 4 == 0:
        # Get the output of the first layer
        layer_outputs.append(inp_to_out([X_test])[0])
        
        # Evaluate model accuracy for this epoch
        test_accuracies.append(model.evaluate(X_test, y_test)[1])

In [ ]:
plot()

If you take a look at the graphs you can see how the neurons are learning to spread out the inputs based on whether they are fake or legit dollar bills. (A single fake dollar bill is represented as a purple dot in the graph) At the start the outputs are closer to each other, the weights are learned as epochs go by so that fake and legit dollar bills get a different, further and further apart output.